Selenium 활용 데이터 수집 - 요일별 네이버 웹툰 목록

selenium\
웹 브라우저를 제어할 수 있는 파이썬 패키지

chromedriver\
selenium과 Google Chrome 브라우저를 연결하는 프로그램\
파이썬 스스로 chromedriver를 내려받도록 하기 위해 chromedriver_autoinstaller  패키지가 필요함

* 셀리니움을 위한 패키지 설치

In [ ]:
# ! pip install --upgrade chromedriver_autoinstaller
# ! pip install --upgrade selenium

* 추가 패키지 설치

In [ ]:
# ChromeDriver 자동 설치 모듈
import chromedriver_autoinstaller
# Chrome 제어용 객체
from selenium import webdriver
# Chrome이 웹 페이지 로딩 완료할 때까지 N초간 대기
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from pandas import DataFrame
# 파이썬 프로그램에 지정된 시간동안 랙을 거는 기능을 위해 사용
import time

* 크롬 브라우저 가동

In [ ]:
# 크롬드라이버 자동 설치
chromedriver_autoinstaller.install()

# 크롬드라이버를 통해 크롬을 실행시킴
# driver 객체는 Chrome 자체
driver = webdriver.Chrome()

# 크롬브라우저가 준비될 때 까지 최대 5초씩 대기
driver.implicitly_wait(5)
# 이후 자동으로 chrome 브라우저 동작(이 브라우저는 건드리지 말것)
# 이 chrome 브라우저에 정보를 기록,
# 데이터를 크롤링 할 수 있도록 유도

In [ ]:
# 요일별 네이버 웹툰에 접근하기 위한 변수값  생성
# URL을 분석하여 얻어낸 값임(노가다 필요)
params = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'dailyPlus']

# 네이버 웹툰의 주소 형식
naverWebtoonUrl  = "https://comic.naver.com/webtoon?tab={0}"
# naverWebtoonUrl

In [ ]:
# 수집된 데이터 결과가 누적될 빈 리스트 생성
naverWebtoolData = []

# 요일별 반복 생성

for p in params:
    # 특정 요일의 네이버 웹툰 페이지
    url = naverWebtoonUrl.format(p)
    # print(url)

    # url을 크롬브라우저로 방문시킴
    driver.get(url)
    # 페이지가 이동할 때마다 시간을 설정해 로딩 시간 제공

    time.sleep(1)

    # 브라우저에 표시되는 전체 코드를 추출(bs4 객체로 변환)
    soup = BeautifulSoup(driver.page_source)
    # print(soup)

    # 자식 연산자를 사용해 class 내부의 class
    # 개발자 도구를 사용해 직접 class를 확인하며 추출
    # class 추출 시 '.'을 붙일 것

    # 웹툰이 표시되는 부분만 추출
    webtoolList = soup.select(".ContentList__content_list--q5KXY > .item")
    # print(webtoolList)

    # 추출된 웹툰 목록 수 만큼 반복
    for w in webtoolList:
        # 포스터 URL
        poster = w.select(".Poster__image--d9XTI")

        # # 가져온 이미지가 존재, src 속성이 있는 경우
        if poster and "src" in poster[0].attrs:
            posterValue = poster[0].attrs['src']
        else:
            posterValue = None

        # print(posterValue)


        # 웹툰의 URL 추출(클릭 시 해당 웹툰 페이지 이동 url 포함 데이터)
        url = w.select(".Poster__link--sopnC")

        # 내부 url 진입은 공통 url을 제외 가능
        # 외부에서 url 진입은 풀 url을 요구
        if url and "href" in url[0].attrs:
            urlValue = url[0].attrs["href"]
            if urlValue.find("https://comic.naver.com/") == -1:
                url = "https://comic.naver.com/"+urlValue
        else:
            urlValue=None
        # print(urlValue)


        # 웹툰 제목 추출
        # 자식 연산자를 사용해 class 내부의 text class 추출
        title = w.select(".ContentTitle__title--e3qXt > .text")

        if title:
            # strip을 사용해 필요없는 값 제거(공백 등)
            titleValue = title[0].text.strip()
        else:
            # 제목이 없는 경우
            titleValue : None

        # print(titleValue)


        # 작가 이름 추출
        author = w.select(".ContentAuthor__author--CTAAP")
        if author:
            authorVlaue = author[0].text.strip()
        else:
            authorVlaue = None

        # print(authorVlaue)


        # 별점 추출
        rating = w.select(".Rating__star_area--dFzsb > .text" )

        if rating:
            ratingValue = rating[0].text.strip()
        else:
            ratingValue = None

        # print(ratingValue)

        
        # 추출 데이터 병합
        resultDic = {"요일" : p, "제목" : titleValue, "작가" : authorVlaue, "별점" : ratingValue}
        # print(resultDic)
        naverWebtoolData.append(resultDic)

In [ ]:
# naverWebtoolData

In [ ]:
# 엑셀로 저장
df = DataFrame(naverWebtoolData)
df.to_excel("요일별_네이버_웹툰.xlsx", index=False)
df